In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.special import factorial, comb
from scipy.special import gamma as gamma_func
from scipy.special import beta as beta_func
from scipy.stats import gamma, beta, binom

from cubature import cubature

from utils import color_cycle

# Lecture 1: Probability, entropy and inference

* **1.1 Bayesian inference**
    * Probability and Bayes rule
    * Inference with discrete random variables
    * Inference with continuous random variables

MacKay, **Chapter 2** (sections 2.1, 2.1, 2.3)

# Probability: the <em>forward</em> way

You can think of conditional probabilities as "Forward probabilities", i.e. the probabilities of possible outcomes given a probability model

$$p(x|f) \rightarrow p(\text{outcome}|f)$$

## Exercise (Mackay Ex 2.4)

Urn with $B$ black balls and $W$ white balls, with total $K=W+B$. 

Draw a ball from the urn $N$ times, with replacement.

What is the probability to draw $N_B$ black balls?

In [ ]:
# urn composition
W = 8
B = 2
K = W + B

# prob of extracting a black ball
f = B / K

# total number of draws
Ns = [5, 400]

# run experiments with N draws a bunch of times
np.random.seed(1)
num_experiment = 1000
Nbs_exp = np.zeros((len(Ns), num_experiment))
for iN, N in enumerate(Ns):
    samples = np.random.rand(num_experiment, N) <= f
    Nbs_exp[iN] = samples.sum(1)

plt.figure(figsize=(10,4))
for iN, N in enumerate(Ns):
    plt.subplot(1,2,iN+1)
    plt.title(f"N={Ns[iN]}")
    plt.hist(Nbs_exp[iN], density=False, histtype="bar");
    plt.xlabel('Nb')
    plt.ylabel('# Nb')
plt.tight_layout();

### Solution

Define $f=B/K$ the fraction of black balls in the urn. Then 
$$
p(N_B)=\binom{N}{N_N}f^{N_B}(1-f)^{N-N_B}
$$

Expected value:
$$
\mathbb{E} N_B=\sum_{N_B=0}^N p(N_B)N_B=\ldots=Nf\qquad
$$

Variance:
$$
\mathbb{V} N_B = \sum_{N_B=0}^N p(N_B)(N_B-Nf)^2=\ldots=Nf(1-f)
$$

In [ ]:
# visualize solution
plt.figure(figsize=(12,4))
for iN, N in enumerate(Ns):
    # compute mean and std
    mean, std = f * N, np.sqrt(f * (1 - f) * N)
    # compute probabilities and compare to experimental frequencies
    Nbs = np.arange(N)
    ps = comb(N, Nbs) * f**Nbs * (1-f)**(N-Nbs)
    plt.subplot(1,2,iN+1)
    plt.title("N={:d}, Nb = {:.2g} $\pm$ {:.2g}".format(Ns[iN], mean, std))
    plt.hist(Nbs_exp[iN], bins=Nbs-0.5, density=True, alpha=0.5);
    plt.plot(Nbs, ps, '.-')
    plt.xlabel('Nb')
    plt.ylabel('# Nb')
plt.tight_layout();

# Probability: the <em>inverse</em> way</center>

When faced with the occurrence of an **unknown event**, we'd like to compute "Inverse Probabilities": given a specific outcome, what is the probability that this has been generated by a model with parameters $f$?

$$
\text{outcome} \rightarrow p(f|\text{outcome}) 
$$

**Solution**: use Bayes' theorem

$$
p(f|\text{outcome}) = \frac{p(\text{outcome}|f) p(f)}{p(\text{outcome})}
$$

## Exercise (Mackay Ex 2.6)

There are eleven urns labelled by $u=0,\ldots,10$, each containing ten balls. Urn $u$ contains $u$ black balls and $10 − u$ white balls. Fred selects an urn $u$ at random and draws $N$ times with replacement from that urn, obtaining $N_B$ blacks and $N − N_B$ whites.

Fred’s friend, Bill, looks on. If after $N = 10$ draws $N_B = 3$ blacks have been drawn, what is the probability that the urn Fred is using is urn $u$, from Bill’s point of view? (Bill doesn’t know the value of $u$.)

In [ ]:
# set urn composition and compute probability for each urn
num_urns = 11
N = 10 # total number of draws

# compute probabilities for each urn
fus = np.arange(num_urns) / (num_urns - 1)

# run a bunch of experiments
num_experiment = 100000

np.random.seed(1)
uNbs = np.zeros((num_experiment, 2))
for iexp in range(num_experiment):
    u = np.random.randint(num_urns) # select an urn at random
    samples = np.random.rand(N) <= fus[u] # draw N times from urn u
    uNbs[iexp] = u, samples.sum() # store urn number and number of black balls

bins = (np.arange(num_urns + 1) - 0.5, np.arange(N + 1 + 1) -0.5)
joint_frequencies, xedges, yedges = np.histogram2d(uNbs[:,0], uNbs[:,1], density=True, bins=bins)
plt.imshow(joint_frequencies);
plt.xlabel('Nb')
plt.ylabel('u');

### Solution

We treat $u$ and $N_B$ as a random variables. We'after

$$p\left(u|N_{B},N\right)$$

First of all the choice of the urn does not depend on the outcome:
$$
p(u|N)=p(u)=\frac{1}{11}
$$

From the joint
$$
p(u,N_B|N)=p(N_B|u,N)p(u|N)
$$

we compute the posterior:
$$
p(u|N_B,N)=\frac{p(N_B|u,N)p(u|N)}{p(N_B|N)}
$$

where:
$$
p(N_B|u,N)=\binom{N}{N_B}f_u^{N_B}(1-f_u)^{N-N_B}\qquad f_u=\frac{u}{10}
$$

All in all we have:
$$
p(N_B|N)=\sum_{u=0}^{10} p(u)p(N_B|u,N)=\frac{1}{11}\binom{N}{N_B}\sum_{u=0}^{10} f_u^{N_B}(1-f_u)^{N-N_B}
$$

##### Final expression:

$$
p(u|N_B,N)=\frac{f_u^{N_B}(1-f_u)^{N-N_B}}{\sum_{u'=0}^{10}f_{u'}^{N_B}(1-f_{u'})^{N-N_B}}
$$

### Visualizing the solution

Define function to compute conditional probability of $N_B$ given $u$ and $N$

In [5]:
def pNb_uN(Nb, u, N, fus = None):
    # default to 11 urns
    if fus is None:
        fus = np.arange(11) / 10
    # select urn
    fu = fus[u]
    # compute likelihood for Nb black balls in N tosses
    p = comb(N, Nb) * fu**Nb * (1-fu)**(N-Nb)
    return p

Compute joint probability of urn $u$ and observed black balls $N_B$

In [ ]:
# compute prob of choosing an urn
pu_N = 1 / num_urns

# compute joint over urn and tosses
puNb_N = np.zeros((num_urns, N + 1))
for u in range(num_urns):
    for Nb in range(N + 1):
        puNb_N[u,Nb] = pu_N * pNb_uN(Nb, u, N, fus=fus)

plt.figure(figsize=(10,4))
plt.subplot(121)
plt.imshow(puNb_N);
plt.xlabel('Nb')
plt.ylabel('u');

plt.subplot(122)
for u in range(num_urns):
    line = plt.plot(joint_frequencies[u], '.:', label="freq" if u == 0 else None);
    plt.plot(puNb_N[u], '.-', c=line[0].get_color(), label="prob" if u == 0 else None);
plt.legend()
plt.xlabel('Nb')
plt.ylabel('freq of (u,Nb), p(u,Nb)')

plt.tight_layout();

Gather observation $N_B$

In [7]:
Nb = 3

and compute posterior urns $u$

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(131)
plt.imshow(puNb_N);
plt.plot()
plt.xlabel('Nb')
plt.ylabel('u')
# select un-normalized conditional from joint
plt.vlines(x=Nb, ymin=0, ymax=num_urns-1, color='red')

plt.subplot(132)
# visualize un-normalized conditional from joint
pu_NbN = puNb_N[:,Nb].copy()
plt.plot(pu_NbN, '.-', c="red");
plt.bar(np.arange(num_urns), height=joint_frequencies[:,Nb], alpha=0.5)
plt.xlabel('u')
plt.ylabel('p(u,Nb|N)')

plt.subplot(133)
# normalize and plot posterior
renorm = pu_NbN.sum()
pu_NbN /= renorm
plt.plot(pu_NbN, '.-');
plt.xlabel('u')
plt.ylabel('p(u|Nb,N)')

plt.tight_layout();

Take a moment to compare `pu_NbN` with the table in Mackay Fig. 2.6.

# Bayesian Inference

Bayesian inference produces **posterior probabilities** of events: we cannot know which urn was selected, but we have a distribution over $u$ that we can use to make decision on further events.

In general, given a model parametrized by $\theta$, the procedure is:
1. Collect data;
2. Specify the prior over models $p(\theta)$;
3. Compute the likelihood of the observed data under the model with parameters $\theta$: $p(\text{data}|\theta)$;
4. Compute the posterior using Bayes' rule:

$$
p(\theta|\text{data})=\frac{p(\text{data}|\theta)p(\theta)}{p(\text{data})}\qquad p(\text{data})=\int d\theta p(\text{data}|\theta)p(\theta)
$$

## Exercise (Mackay Ex 2.6, continued)

Fred draws another ball from the same urn. Given the previous $N$ observations, what is the probability that the ball is black?

### Solution

We (Bill) don't known $u$ but we can sum over the posterior probabilities we just obtained:
$$
p(\text{black}|N_B=3, N=10)=\sum_{u=0}^{10} p(\text{black}|u,N_B,N)p(u|N_B,N)
$$

and since the probability $p(u|N_B,N)$ of drawing from $u$ does not depend on previous observations we get:

$$
p(\text{black}|N_B=3, N=10)=\sum_{u=0}^{10} f_u p(u|N_B,N)
$$

In [9]:
p_new_black = (fus * pu_NbN).sum()
print("p(black|Nb=3, N=10) = ", p_new_black)

p(black|Nb=3, N=10) =  0.3330377815118737


Compare with prediction from most likely urn:

In [10]:
most_likely_urn = np.argmax(pu_NbN)
print("most likely urn: ", most_likely_urn)
print(f"p(black|u={most_likely_urn}) = {fus[most_likely_urn]}")

most likely urn:  3
p(black|u=3) = 0.3


## Exercise (Mackay Example 2.7 and Exercise 2.8)

A (bent) coin has an unknown probability $f$ to come up head. We toss the coin $N$ times and get $N_H$ times head.

What is $f$?

### Solution

Again we treat $f$ as a random variable. Assume a prior over $f$: $p(f)$ is our (subjective) prior belief in the value of $f$.

Given $f$ and $N$, we know the **likelihood** of the observation: 
$$
p(N_H|f,N)=\binom{N}{N_H}f^{N_H}(1-f)^{N-N_H}
$$

and that's how they look like for different $f$'s:

In [ ]:
# set number of tosses
N = 20

fs = [0.2, 0.5, 0.9]

for f in fs:
    Nhs = np.arange(N + 1)
    plt.plot(Nhs, binom.pmf(Nhs, N, f), '.-', label=f"f = {f}");
plt.xlabel("$N_H$")
plt.ylabel(f"p($N_H$|f,N = {N})")

plt.legend();

The posterior reads
$$
p(f|N_H,N)=\frac{p(N_H|f,N)p(f)}{p(N_H|N)}
$$

The **evidence** $p(N_H|N)$, i.e. the normalization of the posterior, must be computed from

$$
\int_0^1 df p(N_H|f,N) p(f)= \binom{N}{N_H} \int_0^1 df f^{N_H}(1-f)^{N-N_H} p(f)
$$


We could of course do it numerically. Assuming $p(f)=1$ for simplicity we get:

In [ ]:
p_Nh_N = np.zeros(N + 1)
for Nh in range(N + 1):
    p_Nh_N[Nh] = cubature(lambda f : binom.pmf(Nh, N, f), 1, 1, [0.], [1.])[0][0]

plt.plot(p_Nh_N, '.-');
plt.xlabel('Nh')
plt.ylabel(f'p(Nh|N={N})');

* Why do you think this makes sense intuitively?

* Can we do it analytically?

## <em>Intermezzo: Gamma and Beta distributions</em>

### A quick reminder of the $\Gamma$ function
$$
\Gamma\left(z\right)=\int_{0}^{\infty}t^{z-1}e^{-t}dt\qquad\Re\left(z\right)>0
$$

$$
\Gamma\left(z+1\right)=z\Gamma\left(z\right)
$$

$$
\Gamma\left(n\right) = \left(n-1\right)!\qquad n\in\mathbb{N}
$$

In [ ]:
maxx = 10
xs = np.arange(1, maxx)
plt.plot(xs, gamma_func(xs), label="Γ(x)");
plt.plot(xs, factorial(xs-1), 'o', label="(x - 1)!")
plt.legend();
plt.yscale('log')

### $\Gamma$ distribution and its cumulants

$$
p\left(x\right)=\frac{b^{a}}{\Gamma\left(a\right)}x^{a-1}e^{-bx}\qquad x>0
$$

### The Beta distribution

The Beta distribution is a probability density over a continuous random variable $x\in [0,1]$ defined by two shape parameters $a,b>0$. 
$$
\text{Beta}(x|a,b)= \frac{\Gamma(a + b)}{\Gamma(a)\Gamma(b)}
x^{a-1}(1-x)^{b-1}\qquad 0\leq x \leq 1
$$

The normalization comes from the formula
$$
\int_0^1 dx x^{a-1}(1-x)^{b-1}=\frac{\Gamma(a)\Gamma(b)}{\Gamma(a+b)}
$$

Mean and variance 
\begin{eqnarray*}
\mathbb{E} x &=& \frac{a}{a+b}\\ 
\mathbb{V} x & = & \frac{ab}{(a+b)^2 (a+b+1)}
\end{eqnarray*}

In [ ]:
# visualize a bunch of shape parameters
pars = [(0.1, 0.1),
        (1., 1.),
        (2., 3.),
        (8., 4.)]

plt.figure(figsize=(12,4))
xs_beta = np.linspace(1e-3, 1-1e-3, 100) # stay just a bit away from boundaries to avoid singularities in numerical computations
for i, (a,b) in enumerate(pars):
    plt.subplot(1,len(pars), i+1)
    plt.plot(xs_beta, beta.pdf(xs_beta, a, b), label=f"a={a}, b={b}");
    norm_beta = gamma_func(a) * gamma_func(b) / gamma_func(a+b)
    fbeta = xs_beta**(a-1) * (1.-xs_beta)**(b-1) / norm_beta
    # plt.plot(xs_beta, fbeta, ':', label="just a check"); # check math is not failing you today
    mean = a/(a+b)
    plt.vlines(x=mean, ymin=0, ymax=beta.pdf(mean, a, b), color='gray', ls=':', label="mean");
    plt.legend();
plt.tight_layout();

### Analytical solution

The posterior
\begin{align}
p(f|N_H,N)=&\frac{ p(N_H|f,N)p(f)}{p(N_H|N)} =\frac{\binom{N}{N_H}}{p(N_H|N)} f^{N_H}(1-f)^{N-N_H}\\
&= \frac{\Gamma(a+b)}{\Gamma(a)\Gamma(b)} f^{a-1}(1-f)^{b-1}
\end{align}
with $N_H=a-1, N-N_H=b-1$ and $p(f)=1$.

From this we can infer that
$$
p(N_H|N)=\binom{N}{N_H} \frac{\Gamma(a)\Gamma(b)}{\Gamma(a+b)}=\frac{N!}{N_H!(N-N_H)!} \frac{N_H! (N-N_H)!}{(N+1)!} =\frac{1}{N+1}
$$
where we used $\Gamma(x)=(x-1)!$ when $x$ is integer. 

Since $p(N_H|N)=\int df p(N_H|f,N)p(f)$ we conclude that **when integrating over all models, the probability of all outcomes are equally likely**.

In [ ]:
# and indeed
plt.hlines(y=1/(N+1), xmin=0, xmax=N, ls=':', color="gray", label="theory")
plt.plot(p_Nh_N, '.-', c=color_cycle[0], alpha=0.5, label="numeric");
plt.xlabel('Nh')
plt.ylabel(f'p(Nh|N={N})')
plt.legend();

## Exercise: the bent coin - reprise

What is the probability to observe another 'head' ($H$) after throwing the coin $N$ times and having observed $N_H$ times 'head'?

### Solution

We don't known $f$ but we can integrate over it and weight each $f$ by its posterior value:
\begin{align}
p(H|N_H,N)&=\int df p(H|f) p(f|N_H,N)\\
&=\int df f \text{Beta}(f|a=N_H+1,b=N-N_H+1) = \frac{a}{a+b} = \frac{N_H+1}{N+2}
\end{align}

Suppose $N=N_H=1$.

Naive answer is $f=1$ and thus $p(H|f)=1$.

Bayesian answer is $p(f|N_H,N)=(N+1)f^1 (1-f)^0 = 2f$ and $p(H|N_H,N)=\frac{N_H+1}{N+2}=\frac{2}{3}$

# <center> Assignments </center>

#### Ex 1.1 (Mackay Ex 2.10)

Urn A contains three balls: one black, and two white; urn B contains three balls: two black, and one white. One of the urns is selected at random and one ball is drawn. The ball is black. What is the probability that the selected urn is urn A?

#### Ex 1.2
Consider $11$ urns $u_{i}$, $i=0,...,10$, each with $10$ balls.
Urn $u_{i}$ has $i$ black balls and $10-i$ white balls. Select
one urn at random, and draw $N$ times with replacement from that
urn. Suppose that the outcome after $N=10$ draws is that the number
of black balls that have been drawn is even. What is the probability
that urn $u_{i}$ was selected?

#### Ex 1.3

Suppose we are told that data are drawn from an oracle that is a Gaussian
distribution with $\sigma=1$ and we believe that $\mu$ can have
any value with equal probability.
* The oracle produces one data point with value $x$. What can we infer
about $\mu$?
* The oracle produces a set of $N$ data points with values $x_{i},...,x_{N}$.
What can we infer about $\mu$ ? Hint: use the fact that the data
points are produced independently. What is the probability that the
data set is drawn from the oracle? Show that the result is Gaussian
in $\mu$ with mean $\bar{x}=\sum_{i=1}^{N}x_{i}$ and variance $\frac{1}{N}$.